# Data Loading
We will use the data set from "Chem. Sci., 2018, 9, 5152-5159." This data set has a features that correlates with the CO binding energy, and we will reproduce their work with Gaussian process.

In [29]:
import pandas as pd
df = pd.read_csv('cat.csv')
df.head()

,Material,Xm,Xp,WLMTO_d,Wcal_d,d_c,COBE
0,Ag3Ag@Ag,4.44,1.93,31.37,1.10,-3.92,-0.09
1,Ag3Au@Ag,4.74,2.07,31.19,1.13,-3.86,-0.11
2,Ag3Co@Ag,4.40,1.92,30.56,1.19,-3.90,-0.11
3,Ag3Cr@Ag,4.25,1.86,30.58,1.13,-4.03,-0.12
4,Ag3Cu@Ag,4.45,1.92,31.43,1.16,-3.99,-0.08


x and y can be separated from the data as shown below. The optimal binding energy for the CO2 reduction reaction is -0.67 eV. So here we set it as well.

In [23]:
data = df.iloc[:,1:].to_numpy()
x = data[:,:5]
y = data[:,5]
y_optimum = -0.67

# Model Loading
Below shows the model loading. We are setting some hyperparameter for kernel, but it will be optimized together so it does not matter what value we set the hyperparameters for.

In [31]:
import warnings
from sklearn.exceptions import ConvergenceWarning
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
# set up model
kernel = 1.0 * RBF(length_scale=0.5)
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0)

# Homework (a)
Use the random.sample function to randomly select the index of the first data base consisting of 3 data points. Fill inthe blank. Look up the function arguments of the random.sample.

In [25]:
import numpy as np
import random
# our initial data sampling
####################### Fill in here ########################
train_idx = random.sample(list(range(len(x))),3)
####################### Fill in here ########################
x_train = x[train_idx,:]
y_train = y[train_idx]

In [26]:
# fit
gpr.fit(x_train,y_train)

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=0.5), random_state=0)

# Homework (b)
Write function to get the test set that is the data points that are not in the training set. 

In [27]:
def get_test_set(x,y,train_idx):
    ####################### Fill in here ########################
    x_test = x[[i for i in range(len(x)) if i not in train_idx],:]
    y_test = y[[i for i in range(len(x)) if i not in train_idx]]
    ####################### Fill in here ########################
    return x_test,y_test

x_test,y_test = get_test_set(x,y,train_idx)

Below is the code that calculates the mean of the standard deviation of the test points. As you can see, the model is not quite confident about its prediction.

In [28]:
y_test, y_test_std = gpr.predict(x_test,return_std=True)
print(np.mean(y_test_std))

0.6150433396445876


# Homework (c) exploration
The following function below find the data point with the highest uncertainty, and add it to the training set.
Write a one line of code for finding the index of the data point with the highest uncertainty.

In [14]:
def addpoint_exploration(gpr,x,y,train_idx):
    x_test,y_test = get_test_set(x,y,train_idx)
    y_test, y_test_std = gpr.predict(x_test, return_std=True)
    ####################### Fill in here ########################
    train_idx_new = int(np.argmax(y_p_std))
    ####################### Fill in here ########################
    train_idx.append(train_idx_new)

    x_train,y_train = x[train_idx,:], y[train_idx]
    gpr.fit(x_train,y_train)
    
    x_test,y_test = get_test_set(x,y,train_idx)
    y_test_, y_test_std = gpr.predict(x_test, return_std=True)
    print('%.3f'%np.mean(y_p_std),np.mean(np.abs(y_p-y_p_)),len(train_idx))

for _ in range(5):
  addpoint_exploration(gpr,x,y,train_idx)

# Homework (d) exploitation
The following function below find the data point with the y value closest to the optimum, and add it to the training set.
Write a one line of code for finding the index of the data point with the highest uncertainty.
The equation we will use to find the optimal point is the data with y value closest to the optimum, while the uncertainty is also the smallest:

$$
\arg \min_{i\in\chi} [\left | y_{opt} - y_{i}  \right | + \sigma_{i}]
$$


In [ ]:
def addpoint_exploitation(gpr,x,y,train_idx):
    x_test,y_test = get_test_set(x,y,train_idx)
    y_test, y_test_std = gpr.predict(x_test, return_std=True)
    ####################### Fill in here ########################
    distance_from_optimal = np.abs(y_test - y_optimum)+y_test_std
    train_idx_new = int(np.argmin(distance_from_optimal))
    ####################### Fill in here ########################
    train_idx.append(train_idx_new)

    x_train,y_train = x[train_idx,:], y[train_idx]
    best_idx = np.argmin(np.abs(y_train - y_optimum))
    print(y_train[best_idx])

for _ in range(5):
  addpoint_exploitation(gpr,x,y,train_idx)